In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import time
import re

In [2]:
directory = '/projectnb/oleds/jmcneel1/FOR_SHAMA'
exception = ['CREST','VG']
computation_cost = pd.DataFrame(columns=['molecule','conformation','image','core','day','hour','minute','second','milli-second'])
error_list = pd.DataFrame(columns=['molecule','conformation','image','error'])

In [3]:
t = time.time()
for molecule in os.listdir(directory):   #for each cruciform
    #print(molecule)
    molecule_path = os.path.join(directory, molecule)
    
    for conformation in os.listdir(molecule_path):   #for each conformation
        #print(conformation)
        
        if conformation not in exception:
            conformation_path = os.path.join(molecule_path, conformation,'GROUND')
            inp_path = os.path.join(conformation_path,'ground.inp')
            out_path = os.path.join(conformation_path,'ground.out')
            
            if Path(inp_path).is_file() == True:
                with open(inp_path,'rt') as inp:
                    inp_line = inp.readlines()[2]
                    cores = inp_line.split()[2]   #number of cores
            else:
                cores = '-'
                error_list.loc[len(error_list)] = [molecule,conformation,'-','inp not found']
            
            if Path(out_path).is_file() == True:
                with open(out_path,'rt') as out:
                    out_line = out.readlines()[-1]
                    if out_line == '\n':
                        run_time = [np.nan]*5   #aborting the run
                        error_list.loc[len(error_list)] = [molecule,conformation,'-','aborting the run']
                    else:
                        run_time = out_line.split()[3:13:2]   #running time
            else:
                run_time = [np.nan] * 5
                error_list.loc[len(error_list)] = [molecule,conformation,'-','out not found']
            
            computation_cost.loc[len(computation_cost)] = [molecule,conformation,'-',cores]+run_time
            
            for image in os.listdir(conformation_path):
                image_path = os.path.join(conformation_path,image)
                if os.path.isdir(image_path):
                    sub_inp_path = os.path.join(image_path,'ground.inp')
                    sub_out_path = os.path.join(image_path,'ground.out')
                    
                    if Path(sub_inp_path).is_file() == True:
                        with open(sub_inp_path,'rt') as inp:
                            inp_line = inp.readlines()[2]
                            cores = inp_line.split()[2]   #number of cores
                    else:
                        cores = '-'
                        error_list.loc[len(error_list)] = [molecule,conformation,image,'inp not found']
                    if Path(sub_out_path).is_file() == True:
                        with open(sub_out_path,'rt') as out:
                            out_line = out.readlines()[-1]
                            if out_line == '\n':
                                run_time = [np.nan]*5   #aborting the run
                                error_list.loc[len(error_list)] = [molecule,conformation,image,'aborting the run']
                            else:
                                run_time = out_line.split()[3:13:2]   #running time
                    else:
                        run_time = [np.nan] * 5
                        error_list.loc[len(error_list)] = [molecule,conformation,image,'out not found']
                    
                    computation_cost.loc[len(computation_cost)] = [molecule,conformation,image,cores]+run_time
        
        elif conformation == 'CREST':
            conformation_path = os.path.join(molecule_path, conformation)
            out_path = ''
            for item in os.listdir(conformation_path):
                if item.endswith('.out'):
                    out_path = os.path.join(conformation_path,item)
                    break
            if Path(out_path).is_file() == True:
                with open(out_path,'rt') as out:
                    out_line = out.readlines()[-3]
                    hour, minute, second = re.findall(r'\d+', out_line)
                    run_time = [0,hour,minute,second,0]
            else:
                run_time = [np.nan] * 5
                error_list.loc[len(error_list)] = [molecule,conformation,'-','out not found']
            computation_cost.loc[len(computation_cost)] = [molecule,conformation,'-',cores]+run_time
            
        else:
            conformation_path = os.path.join(molecule_path, conformation)
            out_path = os.path.join(conformation_path,'vg.out')
            inp_path = os.path.join(conformation_path,'vg.inp')
            
            if Path(inp_path).is_file() == True:
                with open(inp_path,'rt') as inp:
                    inp_lines = inp.readlines()
                    inp_split = inp_lines[2].split()
                    if len(inp_split)==4:
                        cores = inp_split[2]
                    else:
                        inp_split = inp_lines[3].split()
                        cores = inp_split[2]
            else:
                cores = '-'
                error_list.loc[len(error_list)] = [molecule,conformation,'-','inp not found']
            
            if Path(out_path).is_file() == True:
                with open(out_path,'rt') as out:
                    out_line = out.readlines()[-1]
                    if out_line == '\n':
                        run_time = [np.nan]*5   #aborting the run
                        error_list.loc[len(error_list)] = [molecule,conformation,'-','aborting the run']
                    else:
                        run_time = out_line.split()[3:13:2]   #running time
            else:
                run_time = [np.nan] * 5
                error_list.loc[len(error_list)] = [molecule,conformation,'-','out not found']
print(time.time()-t)

16.419729232788086


In [4]:
error_list

,molecule,conformation,image,error
0,8B,19,-,aborting the run


In [5]:
df = computation_cost.sort_values(by=['molecule','conformation']).dropna()
df.to_csv('/projectnb/oleds/Yuning Pan/computation_cost(James).csv', index=False)

In [6]:
df = pd.read_csv (r'/projectnb/oleds/Yuning Pan/computation_cost(James).csv')
df

,molecule,conformation,image,core,day,hour,minute,second,milli-second
0,1,1,-,16,0,0,10,10,175
1,1,10,-,16,0,0,13,55,217
2,1,11,-,16,0,0,13,10,770
3,1,12,-,16,0,0,28,20,968
4,1,2,-,16,0,0,8,29,904
...,...,...,...,...,...,...,...,...,...
751,TT,5,-,16,0,0,47,59,897
752,TT,6,-,16,0,0,36,3,444
753,TT,7,-,16,0,0,35,56,884
754,TT,8,-,16,0,0,32,9,930


In [7]:
df[df['core']!='16']

,molecule,conformation,image,core,day,hour,minute,second,milli-second
726,SM4,1,-,vdw_gaussian,0,6,4,35,748


In [8]:
from datetime import timedelta

In [9]:
df.dtypes

molecule        object
conformation    object
image           object
core            object
day              int64
hour             int64
minute           int64
second           int64
milli-second     int64
dtype: object

In [10]:
cost_per_mol = pd.DataFrame(columns = ['molecule','days','hours','minutes','seconds','milli-seconds'])
name = df['molecule'][0]
sum_d, sum_h, sum_m, sum_s, sum_ms = 0, 0, 0, 0, 0
for i in range(len(df)):
    if df.iloc[i,0] != name or i == len(df)-1:
        cost_per_mol.loc[len(cost_per_mol)] = [name, sum_d, sum_h, sum_m, sum_s, sum_ms]
        name = df.iloc[i,0]
        sum_d, sum_h, sum_m, sum_s, sum_ms = df.iloc[i,4], df.iloc[i,5], df.iloc[i,6], df.iloc[i,7], df.iloc[i,8]
    else:
        sum_d += df.iloc[i,4]
        sum_h += df.iloc[i,5]
        sum_m += df.iloc[i,6]
        sum_s += df.iloc[i,7]
        sum_ms += df.iloc[i,8]

In [11]:
second, cost_per_mol['milli-seconds'] = cost_per_mol['milli-seconds']//1000, cost_per_mol['milli-seconds']%1000
minute, cost_per_mol['seconds'] = (cost_per_mol['seconds']+second)//60, (cost_per_mol['seconds']+second)%60
hour, cost_per_mol['minutes'] = (cost_per_mol['minutes']+minute)//60, (cost_per_mol['minutes']+minute)%60
cost_per_mol['hours'] += hour

In [12]:
hours = (cost_per_mol['minutes']*60 + cost_per_mol['seconds'])/3600
df1 = pd.DataFrame(columns=['molecule','hours'])
df1['molecule'] = cost_per_mol['molecule']
df1['hours'] = cost_per_mol['hours'] + hours

In [13]:
df1.to_csv('/projectnb/oleds/Yuning Pan/cost_per_mol(James).csv', index=False)

In [14]:
error_list.to_csv('/projectnb/oleds/Yuning Pan/error_list(James).csv', index=False)

In [15]:
error_list.shape

(1, 4)